In [2]:
from azureml.core import Workspace

# Enter your workspace subscription, resource group, name, and region.
subscription_id = "cc7f94ab-a419-4d58-9edb-a03f4a70b7bc" #you should be owner or contributor
resource_group = "synapse" #you should be owner or contributor
workspace_name = "mlsynapsedemopackt" #your workspace name
workspace_region = "West US" #your region

ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

In [3]:
blob_account_name = "azureopendatastorage"
blob_container_name = "nyctlc"
blob_relative_path = "yellow"
blob_sas_token = r""

# Allow Spark to read from Blob remotely
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name),blob_sas_token)

# Spark read parquet, note that it won't load any data yet by now
df = spark.read.parquet(wasbs_path)

In [4]:
# Create an ingestion filter
start_date = '2015-01-01 00:00:00'
end_date = '2015-12-31 00:00:00'

filtered_df = df.filter('tpepPickupDateTime > "' + start_date + '" and tpepPickupDateTime< "' + end_date + '"')

filtered_df.describe().show()

+-------+------------------+------------------+------------------+------------+------------+------------------+-----------------+------------------+------------------+------------------+---------------+-------------------+------------------+-------------------+--------------------+--------------------+------------------+------------------+------------------+---------+------------------+
|summary|          vendorID|    passengerCount|      tripDistance|puLocationId|doLocationId|          startLon|         startLat|            endLon|            endLat|        rateCodeId|storeAndFwdFlag|        paymentType|        fareAmount|              extra|              mtaTax|improvementSurcharge|         tipAmount|       tollsAmount|       totalAmount|   puYear|           puMonth|
+-------+------------------+------------------+------------------+------------+------------+------------------+-----------------+------------------+------------------+------------------+---------------+------------------

In [5]:
from datetime import datetime
from pyspark.sql.functions import *

# To make development easier, faster and less expensive down sample for now
sampled_taxi_df = filtered_df.sample(True, 0.001, seed=1234)

taxi_df = sampled_taxi_df.select('vendorID', 'passengerCount', 'tripDistance',  'startLon', 'startLat', 'endLon' \
                                , 'endLat', 'paymentType', 'fareAmount', 'tipAmount'\
                                , column('puMonth').alias('month_num') \
                                , date_format('tpepPickupDateTime', 'hh').alias('hour_of_day')\
                                , date_format('tpepPickupDateTime', 'EEEE').alias('day_of_week')\
                                , dayofmonth(col('tpepPickupDateTime')).alias('day_of_month')
                                ,(unix_timestamp(col('tpepDropoffDateTime')) - unix_timestamp(col('tpepPickupDateTime'))).alias('trip_time'))\
                        .filter((sampled_taxi_df.passengerCount > 0) & (sampled_taxi_df.passengerCount < 8)\
                                & (sampled_taxi_df.tipAmount >= 0)\
                                & (sampled_taxi_df.fareAmount >= 1) & (sampled_taxi_df.fareAmount <= 250)\
                                & (sampled_taxi_df.tipAmount < sampled_taxi_df.fareAmount)\
                                & (sampled_taxi_df.tripDistance > 0) & (sampled_taxi_df.tripDistance <= 200)\
                                & (sampled_taxi_df.rateCodeId <= 5)\
                                & (sampled_taxi_df.paymentType.isin({"1", "2"})))
taxi_df.show(10)

+--------+--------------+------------+------------------+------------------+------------------+------------------+-----------+----------+---------+---------+-----------+-----------+------------+---------+
|vendorID|passengerCount|tripDistance|          startLon|          startLat|            endLon|            endLat|paymentType|fareAmount|tipAmount|month_num|hour_of_day|day_of_week|day_of_month|trip_time|
+--------+--------------+------------+------------------+------------------+------------------+------------------+-----------+----------+---------+---------+-----------+-----------+------------+---------+
|       1|             1|         2.4|-73.98941040039062| 40.77649688720703|-73.96207427978516| 40.77952575683594|          1|      12.0|     2.55|        3|         11|     Friday|          20|      914|
|       2|             2|       16.58|-73.95407104492188| 40.74155044555664| -73.7905044555664|40.643943786621094|          2|      52.0|      0.0|        3|         04|     Sunday

In [6]:
# Random split dataset using Spark, convert Spark to Pandas
training_data, validation_data = taxi_df.randomSplit([0.8,0.2], 223)

In [7]:
from azureml.core import Workspace

# Enter your workspace subscription, resource group, name, and region.
subscription_id = "cc7f94ab-a419-4d58-9edb-a03f4a70b7bc" #you should be owner or contributor
resource_group = "synapse" #you should be owner or contributor
workspace_name = "mlsynapsedemopackt" #your workspace name
workspace_region = "West US" #your region

ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

In [8]:
import pandas 
from azureml.core import Dataset

# Get the Azure Machine Learning Default Datastore
datastore = ws.get_default_datastore()
training_pd = training_data.toPandas().to_csv('training_pd.csv', index=False)

# Convert into Azure Machine Learning Tabular Dataset
datastore.upload_files(files = ['training_pd.csv'],
                       target_path = 'train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)
dataset_training = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train-dataset/tabular/training_pd.csv')])

Uploading an estimated of 1 files
Uploading training_pd.csv
Uploaded training_pd.csv, 1 files out of an estimated total of 1
Uploaded 1 files

In [9]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 10,
    "experiment_timeout_minutes": 30,
    "enable_early_stopping": True,
    "primary_metric": 'r2_score',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 2}

In [10]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',
                             debug_log='automated_ml_errors.log',
                             training_data = dataset_training,
                             spark_context = sc,
                             model_explainability = False, 
                             label_column_name ="fareAmount",**automl_settings)

In [11]:
from azureml.core.experiment import Experiment

# Start an experiment in Azure Machine Learning
experiment = Experiment(ws, "aml-synapse-regression")
tags = {"Synapse": "regression"}
local_run = experiment.submit(automl_config, show_output=True, tags = tags)

# Use the get_details function to retrieve the detailed output for the run.
run_details = local_run.get_details()

Running on ADB.
Running an experiment on spark cluster: aml-synapse-regression.

Parent Run ID: AutoML_19315a78-1c98-4692-9cb5-4b78dd7fc9f2


****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

*************************

In [12]:
# Get best model
best_run, fitted_model = local_run.get_output()

[21:11:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

In [13]:
# Test best model accuracy
validation_data_pd = validation_data.toPandas()
y_test = validation_data_pd.pop("fareAmount").to_frame()
y_predict = fitted_model.predict(validation_data_pd)

In [14]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Calculate Root Mean Square Error
y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))

print("Root Mean Square Error:")
print(rmse)

Root Mean Square Error:
2.0412425592505086

In [15]:
# Calculate MAPE and Model Accuracy 
sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals

print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)

Model MAPE:
0.033843996823639026

Model Accuracy:
0.966156003176361

In [16]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# Calculate the R2 score using the predicted and actual fare prices
y_test_actual = y_test["fareAmount"]
r2 = r2_score(y_test_actual, y_predict)

# Plot the Actual vs Predicted Fare Amount Values
plt.style.use('ggplot')
plt.figure(figsize=(10, 7))
plt.scatter(y_test_actual,y_predict)
plt.plot([np.min(y_test_actual), np.max(y_test_actual)], [np.min(y_test_actual), np.max(y_test_actual)], color='lightblue')
plt.xlabel("Actual Fare Amount")
plt.ylabel("Predicted Fare Amount")
plt.title("Actual vs Predicted Fare Amont R^2={}".format(r2))
plt.show()